[reference link for bootstrapping](https://www.youtube.com/watch?v=Om5TMGj9td4&list=PLqzoL9-eJTNDp_bWyWBdw2ioA43B3dBrl&index=5)

In [2]:
# Data Prepare and Bootstrapping func

import pandas as pd

data = pd.read_csv('ChickData.csv')

mm = data.loc[data['feed'] == 'meatmeal', 'weight']
cs = data.loc[data['feed'] == 'casein', 'weight']
# print (mm.shape, cs.shape)
mm_mean = mm.mean()
cs_mean = cs.mean()

# print (mm_mean, cs_mean)

mm_median = mm.median()
cs_median = cs.median()
# print (mm_median - cs_median)

import numpy as np

class Bootstrapping(): 
    def __init__(self, sample, b = 10000): 
        self.b = b
        self.sample = sample

    def simulate(self): 
        self.b_resample = None

        for _ in range(self.b): 
            temp = np.random.choice(self.sample, len(self.sample), replace = True) 
            
            if self.b_resample is None: 
                self.b_resample = temp
            else: 
                self.b_resample = np.vstack((self.b_resample, temp))
            
        return self.b_resample
    
    def b_means(self): 
        return np.mean(self.b_resample, axis = 1)

    def b_median(self): 
        return np.median(self.b_resample, axis = 1)

mm_bs = Bootstrapping(mm)
mm_bs.simulate()
mm_b_means = mm_bs.b_means()

cs_bs = Bootstrapping(cs)
cs_bs.simulate()
cs_b_means = cs_bs.b_means()

# Bootstrapping

## Bootstrapping/ Hypothesis P value Definition

Under set of assumptions, what is the probability of getting the observed test statistic or one more extreme, if the null hypothesis is True

For instance, sample 1 and sample 2 mean difference: test_statistic = 46.67. 
If the null hypothesis were true, and test_statistic = 0

P-value = the number of bootstrap test statistics that are greater than the observed test statistic / B(the total number of bootstrap test statistics)

if the P-value = .0832 => 8.32% 

Out of the 10,000(B times) bootstrap test statistics calculated, 832 of them had test statistics greater than the observed one. 

If there is no difference in the mean weights, we would see a test statistic of 46.67 (two sample difference) or more by chance roughly 8.32% of the time. (But actually, when I run it, the percentage is around 50%)

In [3]:
def compare_two_mean_bs(sample_1, sample_2, h1_stats): 
    '''
    sample_1 and sample_2 are after boot strapping
    h1_stats is one value. 
    h0 is sample_1 == sample_2
    h1 is sample_1 - sample_2 > h1_stats
    '''
    diff = sample_1 - sample_2
    h1_stats_np = np.array([h1_stats] * len(diff))
    higher_h1 = diff >= h1_stats_np
    return np.mean(higher_h1)
    

p_value = compare_two_mean_bs(cs_b_means, mm_b_means, cs_mean - mm_mean)
print ('If there is no difference in the mean weights, we would see a test statistic of {:.2f} or more by chance roughly {:.2f}% of the time.'.format(cs_mean - mm_mean, p_value * 100))


If there is no difference in the mean weights, we would see a test statistic of 46.67 or more by chance roughly 49.72% of the time.


## Bootstrapping Confidence Interval

Approaches to Building a Bootstrap Confidence Interval: 

- Percentile Method

    - we have 95% confidence that the true population difference ( <- the sample is to compare two samples' means) in means is somewhere between quantile(sample, .025) and quantile(sample, .975)

- Basic Method

- Normal Method

- Bias-Corrected Method

### Percentile Method

For 95% confidence, it uses the 2.5th and 97.5th percentile of the bootstrap distribution of estimates as the lower and upper bounds of the interval. 

So, it uses the middle 95% of bootstrap estimates (removing the bottom/ top 2.5%) to form the 95% confidence interval. 

If the range 2.5% - 97.5% contains 0, we would say the difference is not statistically significant. 

In [4]:
b_means_diff = cs_b_means - mm_b_means
print ('mean difference 2.5% quantile is: {:.2f}\nmean difference 97.5% quantile is: {:.2f}'.format(np.quantile(b_means_diff, .025), np.quantile(b_means_diff, .975)))

print ()

print ('We are 95% confident that the true population difference in means is somewhere between {:.2f}g and {:.2f}g'.format(np.quantile(b_means_diff, .025), np.quantile(b_means_diff, .975)))


mean difference 2.5% quantile is: -3.40
mean difference 97.5% quantile is: 95.99

We are 95% confident that the true population difference in means is somewhere between -3.40g and 95.99g


？视频里说的是：不是明显，因为diff从负到正
但是hyp看到的是 stats 明显, 如果用t test的话？好奇

In [16]:
from scipy.stats import ttest_ind, ttest_ind_from_stats
# from scipy import stats

tt_t, tt_p = ttest_ind(cs_b_means, mm_b_means, equal_var = False)
# print (tt_t, tt_p)


b_means_diff = cs_b_means - mm_b_means
mean = np.mean(b_means_diff)
std = np.std(b_means_diff)/ (len(b_means_diff) ** .5)
print ('diff mean is: {:.2f}'.format(mean))
print ('diff std is: {:.2f}'.format(std))
print ('diff 95% is: {:.2f}, {:.2f}'.format(mean - 2* std, mean + 2*std))
# print (std)
# print (np.var(b_means_diff) ** .5)

diff mean is: 46.45
diff std is: 0.25
diff 95% is: 45.94, 46.96


# Permutation Tests

Concept is: [reference](https://www.youtube.com/watch?v=rJ3AZCQuiLw&list=PLqzoL9-eJTNDp_bWyWBdw2ioA43B3dBrl&index=6)

- resample all the samples without replace; 

- if you want to compare two samples, the resample will go among all the samples. 

- it could not have the confidence interval, but could have the P value, #perm_ts > obs_ts / total number of the Perm (same as the bootstrap p value definition. )

Why Use Permutation Approach

- Small Sample Size

- Assumptions (for parametric approach) not met

- Test something other than classic approaches comparing Means and Medians, like: if one sample is 10% higher than the other. 

- Difficult to estimate the Standard Error for test statistics

- H0: weight gain same under both feed type

- Test stats: 1. mean difference; 2. median difference; 
